In [14]:
import swagger_client
from swagger_client.models import *
from swagger_client.models import ImportDatasetRequest
import socket
import time

In [11]:
# Setup connection to local FVS api
server = socket.gethostbyname(socket.gethostname())
port = "7761"
version = 'v1.0'

# Create FVS api objects
config = swagger_client.configuration.Configuration()
api_config = swagger_client.ApiClient(config)
gsi_boards_apis = swagger_client.BoardsApi(api_config)
gsi_dataset_apis = swagger_client.DatasetApi(api_config)
gsi_datasets_apis = swagger_client.DatasetsApi(api_config)

# Configure the FVS api
config.verify_ssl = False
config.host = f'http://{server}:{port}/{version}'

# Capture the supplied allocation id
Allocation_id = "fd283b38-3e4a-11eb-a205-7085c2c5e516"

# Set default header
api_config.default_headers["allocationToken"] = Allocation_id

# Print dataset count
print("Getting total datasets...")
dsets = gsi_datasets_apis.controllers_dataset_controller_get_datasets_list(allocation_token=Allocation_id)
print(f"Number of datasets:{len(dsets.datasets_list)}")
loaded = gsi_boards_apis.controllers_boards_controller_get_allocations_list(Allocation_id)
print(f"Number of loaded datasets: {len(loaded.allocations_list[Allocation_id]['loadedDatasets'])}")

Getting total datasets...
Number of datasets:95
Number of loaded datasets: 0


In [9]:
loaded = gsi_boards_apis.controllers_boards_controller_get_allocations_list(Allocation_id)
print(f"Number of loaded datasets: {len(loaded.allocations_list[Allocation_id]['loadedDatasets'])}")
# check loaded dataset count
if len(loaded.allocations_list[Allocation_id]["loadedDatasets"]) > 0:
    # Unloading all datasets
    print("Unloading all loaded datasets...")
    loaded = loaded.allocations_list[Allocation_id]["loadedDatasets"]
    for data in loaded:
        dataset_id = data['datasetId']
        resp = gsi_datasets_apis.controllers_dataset_controller_unload_dataset(
                    UnloadDatasetRequest(allocation_id=Allocation_id, dataset_id=dataset_id), 
                    allocation_token=Allocation_id)
        if resp.status != 'ok':
            print(f"error unloading dataset: {dataset_id}")

    # Getting current number of loaded datasets
    curr = gsi_boards_apis.controllers_boards_controller_get_allocations_list(Allocation_id)
    print(f"Unloaded datasets, current loaded dataset count: {len(curr.allocations_list[Allocation_id]['loadedDatasets'])}")

Number of loaded datasets: 2
Unloading all loaded datasets...
JSON method= POST url= http://192.168.99.40:7761/v1.0/dataset/unload request_body= {"allocationId": "fd283b38-3e4a-11eb-a205-7085c2c5e516", "datasetId": "be9ba766-a6da-4878-9c11-22e88a9bf720", "asyncUnload": false} preload True headers {'allocationToken': 'fd283b38-3e4a-11eb-a205-7085c2c5e516', 'Accept': 'application/json', 'Content-Type': 'application/json', 'User-Agent': 'Swagger-Codegen/1.0.0/python'}
JSON method= POST url= http://192.168.99.40:7761/v1.0/dataset/unload request_body= {"allocationId": "fd283b38-3e4a-11eb-a205-7085c2c5e516", "datasetId": "dbdb2bd0-00d6-4f08-9812-c4d384f60878", "asyncUnload": false} preload True headers {'allocationToken': 'fd283b38-3e4a-11eb-a205-7085c2c5e516', 'Accept': 'application/json', 'Content-Type': 'application/json', 'User-Agent': 'Swagger-Codegen/1.0.0/python'}
Unloaded datasets, current loaded dataset count: 0


In [3]:
ds_file_path = "/mnt/nas1/fvs_benchmark_datasets/deep-10K.npy"
resp = gsi_datasets_apis.controllers_dataset_controller_import_dataset(\
    ImportDatasetRequest(ds_file_path=ds_file_path, train_ind=True, nbits=137), \
        allocation_token=Allocation_id)
train_status = gsi_dataset_apis.controllers_dataset_controller_get_train_status(\
    dataset_id=dataset_id, allocation_token=Allocation_id)
while train_status.dataset_status == "training":
    time.sleep(1)
    train_status = gsi_dataset_apis.controllers_dataset_controller_get_train_status(\
        dataset_id=dataset_id, allocation_token=Allocation_id)
    print(train_status)

JSON method= POST url= http://192.168.99.40:7761/v1.0/dataset/import request_body= {"dsFilePath": "/mnt/nas1/fvs_benchmark_datasets/deep-10K.npy", "searchType": "flat", "trainInd": true, "gridTrain": false, "nbits": 137, "qbits": 768, "targetAccuracy": 100, "mdUnique": false, "convertToDataset": false} preload True headers {'allocationToken': 'fd283b38-3e4a-11eb-a205-7085c2c5e516', 'Accept': 'application/json', 'Content-Type': 'application/json', 'User-Agent': 'Swagger-Codegen/1.0.0/python'}


In [12]:
dataset_id = resp.dataset_id
train_status = gsi_dataset_apis.controllers_dataset_controller_get_train_status(\
    dataset_id=dataset_id, allocation_token=Allocation_id)
train_status

{'dataset_status': 'training'}

In [13]:
train_status.dataset_status

'training'

In [15]:
gsi_datasets_apis.controllers_dataset_controller_load_dataset(\
    dataset_id=resp.dataset_id, allocation_token=Allocation_id)

TypeError: controllers_dataset_controller_load_dataset() missing 1 required positional argument: 'body'